In [1]:
# default_exp recognition_results

In [1]:
from fastai import *
from fastai.vision import *
from pathlib import Path
import cv2

In [2]:
path = Path('../data/recognition/black_people/')
images = []
for i, human_dir in enumerate(path.iterdir()):
    for im_path in human_dir.iterdir():
        images.append( (str(im_path), i) )
black_df = pd.DataFrame(images, columns=('im_path', 'id'))
black_df

,im_path,id
0,../data/recognition/black_people/n007714/0260_...,0
1,../data/recognition/black_people/n007714/0203_...,0
2,../data/recognition/black_people/n007714/0009_...,0
3,../data/recognition/black_people/n007714/0182_...,0
4,../data/recognition/black_people/n007714/0136_...,0
...,...,...
16218,../data/recognition/black_people/n003568/0117_...,49
16219,../data/recognition/black_people/n003568/0028_...,49
16220,../data/recognition/black_people/n003568/0050_...,49
16221,../data/recognition/black_people/n003568/0036_...,49


In [3]:
num_ppl = black_df['id'].max() + 1

In [5]:
df = black_df
test_images = []
num_false_person, num_true_person = 20, 20
for i in progress_bar(range(num_ppl)):
    original_im_path = random.choice( list(df[df['id'] == i]['im_path']) )
    for j in range(num_false_person):
        false_im_path = random.choice( list(df[df['id'] != i]['im_path']) )
        test_images.append( (original_im_path, false_im_path, False) )
    for j in range(num_true_person):
        true_im_path = random.choice( list(df[df['id'] == i]['im_path']) )
        test_images.append( (original_im_path, true_im_path, True) )
test_df = pd.DataFrame(test_images, columns=('imA', 'imB', 'is_same'))
test_df

,imA,imB,is_same
0,../data/recognition/black_people/n007714/0019_...,../data/recognition/black_people/n005203/0263_...,False
1,../data/recognition/black_people/n007714/0019_...,../data/recognition/black_people/n001621/0018_...,False
2,../data/recognition/black_people/n007714/0019_...,../data/recognition/black_people/n008580/0153_...,False
3,../data/recognition/black_people/n007714/0019_...,../data/recognition/black_people/n009152/0190_...,False
4,../data/recognition/black_people/n007714/0019_...,../data/recognition/black_people/n000476/0074_...,False
...,...,...,...
1995,../data/recognition/black_people/n003568/0398_...,../data/recognition/black_people/n003568/0242_...,True
1996,../data/recognition/black_people/n003568/0398_...,../data/recognition/black_people/n003568/0020_...,True
1997,../data/recognition/black_people/n003568/0398_...,../data/recognition/black_people/n003568/0129_...,True
1998,../data/recognition/black_people/n003568/0398_...,../data/recognition/black_people/n003568/0270_...,True


In [6]:
import face_recognition
def is_same_face(imA_path, imB_path, tolerance=0.6):
    known_image = face_recognition.load_image_file(imA_path)
    unknown_image = face_recognition.load_image_file(imB_path)
    biden_encoding = face_recognition.face_encodings(known_image)[0]
    unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
    return face_recognition.compare_faces([biden_encoding], unknown_encoding, tolerance=tolerance)[0]

In [9]:
thresholds = range(0,100,5)

In [7]:
res = []
for t in thresholds:
    t = t/100
    correct_results, errors = [], 0
    for a,b,is_same_true in progress_bar(list(zip(test_df['imA'], test_df['imB'], test_df['is_same']))):
        try:
            is_same_pred = is_same_face(a, b, tolerance=t)
            correct_results.append( (is_same_pred, is_same_true) )
        except:
            errors += 1
            correct_results.append( ('unk', is_same_true) )
    res.append(correct_results)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
for r,t in zip(res, thresholds):
    r = list(filter(lambda x: x[0] != 'unk', r))
    r = [(a[0],b) for a,b in r]
    correct_results = [pred == true for pred, true in r]
    print(t, np.array(correct_results).mean())

0 0.4942938659058488
5 0.4942938659058488
10 0.4942938659058488
15 0.4942938659058488
20 0.4942938659058488
25 0.4950071326676177
30 0.5014265335235378
35 0.5271041369472182
40 0.6005706134094151
45 0.6918687589158345
50 0.8109843081312411
55 0.8858773181169758
60 0.8930099857346647
65 0.8495007132667618
70 0.7710413694721826
75 0.6875891583452212
80 0.6248216833095578
85 0.5699001426533523
90 0.5342368045649073
95 0.514265335235378


In [14]:
correct_results = list(filter(lambda x: x != 'unk', correct_results))

In [15]:
np.array(correct_results).mean()

0.8777292576419214

In [91]:
for i in range(len(test_df)):
    
    break

0


In [43]:
import face_recognition

known_images = [face_recognition.load_image_file(x) for x in [im_pathA1, im_pathA2, im_pathA3]]
unknown_image = face_recognition.load_image_file(im_pathB1)

biden_encodings = [face_recognition.face_encodings(x)[0] for x in known_images]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces(biden_encodings, unknown_encoding)
results

[True, False, False]

In [41]:
 face_recognition.compare_faces??